In [1]:
import pandas as pd
from datetime import datetime

there are date time object in the faults will read the csv again.`

In [5]:
# faults = pd.read_csv("../data/J1939Faults.csv") donot ignore warning low_memory=False as you loose lot of rows.

faults = pd.read_csv("../data/J1939Faults.csv", low_memory=False,
                    parse_dates=['EventTimeStamp', 'LocationTimeStamp'])
faults.shape #(1187335, 20)
faults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187335 entries, 0 to 1187334
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   RecordID               1187335 non-null  int64         
 1   ESS_Id                 1187335 non-null  int64         
 2   EventTimeStamp         1187335 non-null  datetime64[ns]
 3   eventDescription       1126490 non-null  object        
 4   actionDescription      0 non-null        float64       
 5   ecuSoftwareVersion     891285 non-null   object        
 6   ecuSerialNumber        844318 non-null   object        
 7   ecuModel               1122577 non-null  object        
 8   ecuMake                1122577 non-null  object        
 9   ecuSource              1187335 non-null  int64         
 10  spn                    1187335 non-null  int64         
 11  fmi                    1187335 non-null  int64         
 12  active                 11873

In [7]:
faults.head(5)

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25
1,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08
4,5,990416,2015-02-21 11:39:41,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37


In [8]:
faults.isna().sum()

RecordID                       0
ESS_Id                         0
EventTimeStamp                 0
eventDescription           60845
actionDescription        1187335
ecuSoftwareVersion        296050
ecuSerialNumber           343017
ecuModel                   64758
ecuMake                    64758
ecuSource                      0
spn                            0
fmi                            0
active                         0
activeTransitionCount          0
faultValue               1187335
EquipmentID                    0
MCTNumber                      0
Latitude                       0
Longitude                      0
LocationTimeStamp              0
dtype: int64

In [9]:
# as the actionDescription and faultValue  are all NaN values will drop them.

faults = faults.drop(columns = ['actionDescription', 'faultValue'])
faults

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,17,True,2,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25
1,2,990360,2015-02-21 11:34:34,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,False,127,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,True,127,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08
4,5,990416,2015-02-21 11:39:41,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21
1187331,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49


In [13]:
#cleaning the faults data to hav equipmentID less than 5
#faults = 1187335


faults = faults[faults['EquipmentID'].str.len() <=5]
faults.shape #1185166, 18

(1185166, 18)

In [11]:
faults.loc[faults['spn'] == 5246]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
45,46,990931,2015-02-21 12:10:51,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1395,105349612,36.065972,-86.433425,2015-02-21 12:11:27
1918,1919,1007751,2015-02-22 19:44:55,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1395,105349612,36.066203,-86.434814,2015-02-22 19:46:27
2058,2059,1010486,2015-02-23 04:00:21,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,False,1,1395,105349612,36.066666,-86.434537,2015-02-23 01:06:06
2089,2090,1011009,2015-02-23 05:05:44,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,5246,0,True,1,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23
2971,2972,1026305,2015-02-23 15:54:22,NaN,unknown,unknown,unknown,unknown,0,5246,0,True,1,1487,105369355,28.077361,-81.897083,2015-02-23 15:54:58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183032,1244156,121610128,2020-02-19 07:02:33,NaN,05317106*05005224*051718172255*09401583*G1*BDR*,79845785,6X1u13D1500000000,CMMNS,0,5246,0,True,1,1814,105369518,36.067037,-86.434120,2020-02-19 07:03:09
1183684,1244808,121909497,2020-02-21 07:23:44,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,True,1,2211,105329862,36.066296,-86.434305,2020-02-21 07:24:20
1184328,1245452,122305094,2020-02-24 15:28:05,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,False,1,2211,105329862,36.066620,-86.434722,2020-02-24 15:28:01
1184330,1245454,122305096,2020-02-24 15:27:26,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,0,True,1,2211,105329862,36.066620,-86.434722,2020-02-24 15:28:02


Note we need to find the rows which are 0.01 mile from the service stations and drop them.

We do not have the target variable column in the data set so we need to find four groups which never derate, which have 1526 but no 5246,
some with 1526 and 5246 and some with 5246 only.

In [47]:
faults['EquipmentID'].nunique() #1045

1045

In [46]:
faults[(faults['spn'] == 1569) & (faults['fmi'] == 31)] #10717 with and without fmi=31

faults[(faults['spn'] == 1569) & (faults['fmi'] == 31)]['EquipmentID'].nunique() #498

498

In [48]:
faults.loc[~(faults['spn'].isin([1569, 5426]))] #1174449
# faults.loc[~(faults['spn'] .isin([1569, 5426]))]['EquipmentID'].nunique() #1045

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,17,True,2,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25
1,2,990360,2015-02-21 11:34:34,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,False,127,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,True,127,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08
4,5,990416,2015-02-21 11:39:41,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187329,1248453,123901925,2020-03-06 13:44:45,Low (Severity Medium) Engine Coolant Level,04358814*06030918*051718174436*09401683*G1*BDR*,79904453,6X1u13D1500000000,CMMNS,0,111,18,False,93,1886,105351219,39.013472,-77.037685,2020-03-06 13:44:41
1187330,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49


In [45]:
faults[(faults['spn']==5246)] #1189
# faults[(faults['spn']==5246)]['EquipmentID'].nunique() #211

# same number of rows as for 1569 which is 10717 rows × 18 columns

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
45,46,990931,2015-02-21 12:10:51,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1395,105349612,36.065972,-86.433425,2015-02-21 12:11:27
1918,1919,1007751,2015-02-22 19:44:55,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1395,105349612,36.066203,-86.434814,2015-02-22 19:46:27
2058,2059,1010486,2015-02-23 04:00:21,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,False,1,1395,105349612,36.066666,-86.434537,2015-02-23 01:06:06
2089,2090,1011009,2015-02-23 05:05:44,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,5246,0,True,1,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23
2971,2972,1026305,2015-02-23 15:54:22,NaN,unknown,unknown,unknown,unknown,0,5246,0,True,1,1487,105369355,28.077361,-81.897083,2015-02-23 15:54:58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183032,1244156,121610128,2020-02-19 07:02:33,NaN,05317106*05005224*051718172255*09401583*G1*BDR*,79845785,6X1u13D1500000000,CMMNS,0,5246,0,True,1,1814,105369518,36.067037,-86.434120,2020-02-19 07:03:09
1183684,1244808,121909497,2020-02-21 07:23:44,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,True,1,2211,105329862,36.066296,-86.434305,2020-02-21 07:24:20
1184328,1245452,122305094,2020-02-24 15:28:05,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,False,1,2211,105329862,36.066620,-86.434722,2020-02-24 15:28:01
1184330,1245454,122305096,2020-02-24 15:27:26,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,0,True,1,2211,105329862,36.066620,-86.434722,2020-02-24 15:28:02


In [40]:
(10717+1174449 +1189 )

1186355

In [60]:
faults['latA_diff'] = faults['Latitude']-36.0666667
faults['latB_diff'] = faults['Latitude']-35.5883333
faults['latC_diff'] = faults['Latitude']-36.1950
faults['longA_diff'] = faults['Longitude']-(-86.4347222)
faults['longB_diff'] = faults['Longitude']-(-86.4438888)
faults['longC_diff'] = faults['Longitude']-(-83.174722)

faults.head(5)

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,...,MCTNumber,Latitude,Longitude,LocationTimeStamp,latA_diff,latB_diff,latC_diff,longA_diff,longB_diff,longC_diff
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,...,105354361,38.857638,-84.626851,2015-02-21 11:34:25,2.790971,3.269305,2.662638,1.807871,1.817038,-1.452129
1,2,990360,2015-02-21 11:34:34,NaN,unknown,unknown,unknown,unknown,11,629,...,105354361,38.857638,-84.626851,2015-02-21 11:35:10,2.790971,3.269305,2.662638,1.807871,1.817038,-1.452129
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,...,105336226,41.421250,-87.767361,2015-02-21 11:35:26,5.354583,5.832917,5.226250,-1.332639,-1.323472,-4.592639
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,...,105336226,41.421018,-87.767361,2015-02-21 11:36:08,5.354351,5.832685,5.226018,-1.332639,-1.323472,-4.592639
4,5,990416,2015-02-21 11:39:41,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,...,105427130,38.416481,-89.442638,2015-02-21 11:39:37,2.349814,2.828148,2.221481,-3.007916,-2.998749,-6.267916


In [14]:
diagnostic = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")
# diagnostic.shape
# diagnostic.info

In [15]:
diagnostic.head(5)

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [16]:
diagnostic.isna().sum()

Id         0
Name       0
Value      0
FaultId    0
dtype: int64

In [17]:
diagnostic.info

<bound method DataFrame.info of                 Id                      Name        Value  FaultId
0                1                 IgnStatus        False        1
1                2         EngineOilPressure            0        1
2                3      EngineOilTemperature     96.74375        1
3                4        TurboBoostPressure            0        1
4                5                EngineLoad           11        1
...            ...                       ...          ...      ...
12821621  12864020  EngineCoolantTemperature        181.4  1248457
12821622  12864021              ParkingBrake        False  1248457
12821623  12864022    SwitchedBatteryVoltage         14.1  1248457
12821624  12864023               DistanceLtd  28606.65625  1248457
12821625  12864024                LampStatus         1023  1248458

[12821626 rows x 4 columns]>

In [18]:
diagnostic.dtypes

Id          int64
Name       object
Value      object
FaultId     int64
dtype: object

In [19]:
diagnostic['Value'].value_counts().to_frame().reset_index().head(50)
# 12821626 - 1009465

,index,Value
0,0,981492
1,False,793736
2,True,789606
3,100,417610
4,1023,353059
5,1279,314513
6,255,301633
7,32,291368
8,66.48672,278448
9,64.6226,130898


In [54]:
diagnostic['FaultId'].value_counts().to_frame().reset_index()

diagnostic['FaultId'].nunique()

1187335

In [21]:
diagnostic['Name'].value_counts().to_frame().reset_index()

,index,Name
0,LampStatus,1187335
1,IgnStatus,608454
2,EngineRpm,586921
3,IntakeManifoldTemperature,586291
4,EngineOilPressure,586244
5,EngineCoolantTemperature,586071
6,BarometricPressure,585976
7,DistanceLtd,585819
8,EngineLoad,585621
9,FuelRate,585237


In [53]:
diagnostic.pivot(columns='FaultId', index='Id', values=['Name', 'Value'])

C:\Users\upadh\anaconda3\lib\site-packages\pandas\core\reshape\reshape.py:126: RuntimeWarning: overflow encountered in long_scalars
  num_cells = num_rows * num_columns


ValueError: negative dimensions are not allowed

In [49]:
diagnostic.head(5)

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [22]:
service = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")
service.shape #(7124, 14)

C:\Users\upadh\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


(7124, 14)

In [23]:
service.head(5)

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...
